In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"

import torch
import torch.nn as nn
import torch.optim as optim
from transformers import OwlViTForObjectDetection, OwlViTProcessor
from torch.utils.data import Dataset, DataLoader
from torchinfo import summary
from extract_features import get_features
from tqdm.auto import tqdm, trange
import matplotlib.pyplot as plt
import numpy as np
import json

from PIL import Image, ImageDraw

In [ ]:

semi = 2

# NOTE: 1 and 2 OVERFIT massively! Only use with a LOT of data!

# 0 = fully unsupervised (features and bboxes from the model)
# 1 = semi-supervised (handlabeled data and features from the model)
# 2 = fully supervised (handlabeled data and external feature extractor)

if semi == 0:
    classes = [["object", "Can: Sealed cylindrical metal container", "Bottle: Narrow-necked liquid storage container"]]

    features = get_features(texts=classes)

    features_torch = []
    class_ids_torch = []
    patches = []

    # extract features and make tensors
    for feature in features:
        feature_tensor = feature[0].detach().cpu()
        label = feature[1]
        class_id = feature[2]

        features_torch.append(feature_tensor)
        class_ids_torch.append(torch.as_tensor(class_id))
        patches.append(feature[3])

    # stack lists:
    X_data = torch.stack(features_torch)
    y_data = torch.stack(class_ids_torch)

elif semi == 1:
    root_dir = '/caa/Homes01/mburges/viennaup23-hackathon-recycling/backend/data/'
    data_json = json.load(open(root_dir + 'labels.json'))
    image_path = os.path.join(root_dir, 'images')

    with torch.no_grad():
        processor = OwlViTProcessor.from_pretrained("google/owlvit-large-patch14")
        model = OwlViTForObjectDetection.from_pretrained("google/owlvit-large-patch14")
        base_model = model.owlvit

        del(model)

        base_model.eval()
        base_model.to("cuda")

        features_torch = []
        class_ids_torch = []
        for file in tqdm(data_json):
            image = Image.open(os.path.join(image_path, file))

            inputs = processor(images=image, return_tensors="pt").to("cuda")
            image_features = base_model.get_image_features(**inputs).squeeze()

            features_torch.append(image_features.detach().cpu())
            class_ids_torch.append(torch.as_tensor(data_json[file]))

        X_data = torch.stack(features_torch)
        y_data = torch.stack(class_ids_torch)

else:
    root_dir = '/caa/Homes01/mburges/viennaup23-hackathon-recycling/backend/data/'
    data_json = json.load(open(root_dir + 'labels.json'))

    features_path = os.path.join(root_dir, 'features_noclip64')

    features_torch = []
    class_ids_torch = []
    for file in data_json:
        npy_ = file.replace('.png', '.npy')
        features = np.load(os.path.join(features_path, npy_))

        features_torch.append(torch.as_tensor(features))
        class_ids_torch.append(torch.as_tensor(data_json[file]))
    
    # stack lists:
    X_data = torch.stack(features_torch).float()
    y_data = torch.stack(class_ids_torch)

print(X_data.shape)
print(y_data.shape)

In [ ]:
class FloKo(nn.Module):
    def __init__(self):
        super(FloKo, self).__init__()
        self.fc = nn.Linear(64, 32)
        self.dropout_1 = nn.Dropout(p=0.2)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(32, 2)
        self.fc4 = nn.Linear(2, torch.stack(class_ids_torch).max()+1)
        self.sigmoid = nn.Sigmoid()


    def forward(self, x):
        x = self.fc(x)
        x = self.dropout_1(x)
        x = self.relu(x)
        intermediate = self.fc3(x)
        x = self.fc4(intermediate)
        x = self.sigmoid(x)

        return x, intermediate

model = FloKo()

print(summary(model, (64,)))

In [ ]:
# Training loop
num_epochs = 250
model.to("cuda")

# split data into train and test
X_train = X_data[:int(len(X_data)*0.8)]
y_train = y_data[:int(len(y_data)*0.8)]

X_test = X_data[int(len(X_data)*0.8):]
y_test = y_data[int(len(y_data)*0.8):]

# patches_train = patches[:int(len(patches)*0.8)]
# patches_test = patches[int(len(patches)*0.8):]

# Create a TensorDataset
dataset_train = torch.utils.data.TensorDataset(X_train, y_train)
# Create a TensorDataset
dataset_test = torch.utils.data.TensorDataset(X_test, y_test)
# Create a DataLoader
batch_size = 10
dataloader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=1, shuffle=False)


optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()

losses_train, losses_test = [], []
for epoch in trange(num_epochs):
    total_loss_train = []
    total_loss_test = []
    for inputs, targets in dataloader:
        model.train()
        # Zero the gradients
        optimizer.zero_grad()

        inputs = inputs.to("cuda")
        targets = targets.to("cuda")

        # encode targets hot one
        targets = torch.nn.functional.one_hot(targets, num_classes=3).float()
        
        # Forward pass
        output, intermediate = model(inputs)
        
        # Compute the loss
        loss = loss_function(output, target=targets)
        
        # Backward pass
        loss.backward()
        
        # Update the weights
        optimizer.step()
        total_loss_train.append(loss.item())
    for inputs, targets in test_loader:
        model.eval()
        inputs = inputs.to("cuda")
        targets = targets.to("cuda")

        # encode targets hot one
        targets = torch.nn.functional.one_hot(targets, num_classes=3).float()
        
        # Forward pass
        output, intermediate = model(inputs)
        
        # Compute the loss
        loss = loss_function(output, target=targets)
        total_loss_test.append(loss.item())
    losses_train.append(np.mean(total_loss_train))
    losses_test.append(np.mean(total_loss_test))


In [ ]:
# plot loss
plt.plot(losses_train, label='train')
plt.plot(losses_test, label='test')
plt.show()

In [ ]:
# run model over dataset and get features
model.eval()

features = []
class_ids = []
predictions = []

for inputs, labels in tqdm(test_loader):
    inputs = inputs.cuda()
    outputs, intermediate = model(inputs)
    features.append(intermediate.squeeze().detach().cpu().numpy())
    predictions.append(outputs.squeeze().detach().cpu().numpy())
    class_ids.append(labels.detach().cpu().numpy())

features = np.array(features)
class_ids = np.array(class_ids)
predictions = np.argmax(np.array(predictions), axis=1)

print(features.shape)
print(class_ids.shape)
print(predictions.shape)

In [ ]:
# Get weights of last layer
weights = model.fc4.weight.detach().cpu().numpy()

# plot features and labels in matplotlib
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
ax1.set_title('Labels')
ax1.scatter(features[:, 0], features[:, 1], c=class_ids, cmap='coolwarm')
ax2.set_title('Predictions')
ax2.scatter(features[:, 0], features[:, 1], c=predictions, cmap='coolwarm')
#draw lines of weights

plt.show()

### vvv IGNORE vvv

In [ ]:
print(features.shape)
print(class_ids.shape)
print(predictions.shape)
print(len(patches_test))

save_path = "/caa/Homes01/mburges/viennaup23-hackathon-recycling/backend/data/images_and_json_for_UI"

for i in range(len(patches_test)):
    patch = patches_test[i]
    patch.save(os.path.join(save_path, f"patch_{i}.png"))

    # save json
    json_ = {"features": features[i].tolist(), "class_id": class_ids[i].tolist(), "name": f"patch_{i}.png"}
    with open(os.path.join(save_path, f"pred_before_patch_{i}.json"), 'w') as f:
        json.dump(json_, f)

    # save json
    json_ = {"features": features[i].tolist(), "class_id": predictions[i].tolist(), "name": f"patch_{i}.png"}
    with open(os.path.join(save_path, f"pred_after_patch_{i}.json"), 'w') as f:
        json.dump(json_, f)